In [ ]:
'''
The Following code is written for handling Z-scan closed aperture data found from the
Z-scan device at the Nonlinear Optics Laboratory, Department of Physics,
Shahjalal University of Science and Technology, Sylhet-3114, Bangladesh.

This Program will take input from the .csv file generated by the Z-scan device,
and will work on all the .csv files present in the folder. Where multiple files will be present
carrying data for various powers. Then another .csv file will be generated using this program
that will store all the important calculated values and use them for further data representation.

This program is developed by:
Tanver Hossain Refat, M.Sc.(Thesis) student (Session: 2021-22)
Department of Physics, SUST.

Under the supervision of:
Dr. Md. Enamul Haque,
Associate Professor,
Department of Physics, SUST.
'''

import os # for operating system dependent functionality
import glob # for reading all the file names on the folder
import matplotlib.pyplot as plt # data visualization package
import numpy as np # array processing package
import scipy as sp # scientific library
from scipy import stats
from scipy.optimize import curve_fit # for curve fitting
from scipy.stats import sem # for standard error of mean (SEM)
import pandas as pd # data analytic package

# Defining sample names as an icon for easy input and avoiding mistakes.
a, b, c, d, e, f, g, h, i, j, k, l = "AR","DS","CL","DL","OT","D3","VD","CF","DV","DB","HD","DL"  # These are the folder names, also the madeup names for the samples

sample = e # taking input

lmda = 655e-9 #wave length of laser in meter (red)
#F = #focal length of the lens
#D = #beam diameter
#w0 = ((2*lmda*F)/((np.pi)*D)) #formula for beam waist.
w0 = 1.81e-5 #beam waist for the above light in meters
z0 = (np.pi)*(w0*w0)/lmda # Rayleigh range

# Lists to store the calculated values for all the files
nums = [] # power numbers i.e. 189
pwrs = [] # corresponding power for the power number
irdnc = []  # corresponding irradiance for power
delphis = [] # nonlinear phase shift
abs_delphis = [] # absolute values of delphi
delphi_errs = [] # covarience of delphi
Tpvs = [] # delta-T_pv
Zpvs = [] # delta-T_pv
fit = [] # fit percentage of Z-scan

# defining the curve fitting function as "zscan"
def zscan(x,phi):
   T = 1 +(((4*x)/(9+(10*(x**2))+(x**4)))*phi) #closed aparture Z-scan formula
   return T

# File path from computer or drive
filepath = f'/content/drive/MyDrive/Thesis/Z-Scan Data/Table/Calculated/{sample}'

csv_files = glob.glob(os.path.join(filepath, '*_calculated.csv')) # creating a list of all CSV files in the folder

# Loop for automatically reading the files from the folder
for file_path in csv_files:
    df = pd.read_csv(file_path)
    Xax = np.array(df['x'].tolist()) # in meters
    Xcm = np.array([i*100 for i in Xax]) # in centi meters
    Z = np.array([i/z0 for i in Xax])
    Yax = np.array(df['I/I0'].tolist())

    # Extract the number from the filename
    base = os.path.basename(file_path) # Extract the base name of the file
    name, _ = os.path.splitext(base) # the base name without extension (.csv)
    number_str, _ = name.split('_') # Split the name
    num = int(number_str) # Convert string to an integer
    pwr = ((-1467)+(12.6*num)-(0.0242*num*num)) # laser power for power number (Emperical Relation with PWM: exclusive to NLO SUST)
    I0 = (pwr*(0.001))/((np.pi)*(w0*w0))  # irradiance at a particular laser power

    # Using the Scipy curve fit module
    popt, pcov = curve_fit(zscan, Z, Yax) # fitting the "zscan function", finding the optimized parameter 'popt' and its coverience 'pcov'
    Yax2 = zscan(Z, *popt) # getting predicted values from the "zscan" function and naming them as "Yax3"

    phi = popt # getting delta-phi as the optimised parameter from the fit
    delphi = phi[0] # storing the value of phi as delphi
    delphi_err = pcov[0][0]  # getting error from the covariance matrix
    delphi_errs.append(delphi_err)  # append the error to delphi_errs

    Tpv = max(Yax2) - min(Yax2) # measuring delta-T_pv
    Tp = np.argmax(Yax2) # x axis value for fitted transmission peak
    Tv = np.argmin(Yax2) # x axis value for fitted transmission valley
    Zpv = abs(((Z[Tv] - Z[Tp])*z0)) # measuring delta-Z_pv

    # Measuring the R-square (fit) value
    dif = Yax - Yax2 # difference of y axis datapoints for each x axis points
    Tot_ver = np.sum((Yax-np.mean(Yax))**2) # total verience
    Res_ver = np.sum(dif**2) # residual verience
    R_sq = 1 - (Res_ver / Tot_ver) # R-square value
    percent = R_sq*100 # percentage error

    # difference of fit from experimental data
    Y_diff = abs(Yax2-Yax)

    # Creating a list with the calculated values
    nums.append(num) # power number (185-254)
    pwrs.append(pwr) # corresponding power
    irdnc.append(I0) # irradiance at particular power
    delphis.append(delphi) # calculated delphi values
    abs_delphis.append(abs(delphi)) # |calculated delphi values|
    Tpvs.append(Tpv) # vertical difference of peak-valley transmitance
    Zpvs.append(Zpv) # Horizontal difference of peak-valley transmitance
    fit.append(percent) # percentage of Z-scan fit


    # plotting the graph
    plt.figure(figsize=(1.618*5, 5))
    plt.title(r'Closed Aparture Z-scan profile at {:.2f} mW'.format(pwr), fontsize=12) # plot title
    plt.ylabel(r"Normalized Transmitance (I/I$_0$)", fontsize=10) # y-axis label
    plt.xlabel("Sample Position (Z in cm)", fontsize=10) # x-axis label
    plt.xlim(-1, 1) # showing Z-profile for only 2 cm out of 6 cm.
    plt.errorbar(Xcm, Yax, yerr=Y_diff, fmt='.', color='#0C359E', ecolor='lightskyblue', elinewidth=.9, markersize=8, mfc='w', label='Experimental data') # experimental datapoints with error
    plt.plot(Xcm, Yax2, "-", color='crimson', label='Theoretical Fit ({:.2f}%)'.format(percent)) #plotting the theoretical fit line with fit percentage
    plt.text(0.001, 0.95, '$\Delta$ $\phi$: {:.9f}'.format(delphi), transform=plt.gca().transAxes, fontsize=10, color='#0C359E')
    plt.text(0.001, 0.90, '$\Delta$T$_p$$_v$: {:.9f}'.format(Tpv), transform=plt.gca().transAxes, fontsize=10, color='#0C359E')
    plt.text(0.001, 0.85, '$\Delta$Z$_p$$_v$: {:.5f}'.format(Zpv), transform=plt.gca().transAxes, fontsize=10, color='#0C359E')
    plt.legend(fontsize=10, loc="upper right")

    ax = plt.gca() # Get current axes
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(9)  # Set x-limit font size
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(9)  # Set y-limit font size

    # plt.savefig(f'{filepath}/{num}.png', dpi=300) # Saving figure as png
    plt.show()
    plt.clf()

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Power number': nums,
    'P(mW)': pwrs,
    'Irradiance (I0) (W/m^2)': irdnc,
    'delphis': delphis,
    'abs_delphis': abs_delphis,
    'delphi_errs': delphi_errs,
    'Tpvs': Tpvs,
    'Zpvs': Zpvs,
    '%fit': fit
})
# df = df.sort_values('Power number') # Sort the DataFrame by 'Power number'
# df.to_csv(os.path.join(filepath, f'{sample} Measurments.csv'), index=False) # Save the DataFrame as a CSV file

'''
Following part is for "Delphi vs DelTpv graph" and "Power vs Delphi graph"
The Part above this comment can be used as a standalone function for just plotting the Z-profile for all the files.
'''

# Calculate the standard deviation of Tpvs and abs_delphis
Tpvs_sd = np.std(Tpvs)
abs_delphis_sd = np.std(abs_delphis)

# Calculate the standard error of the mean SEM) for Del-T_pv and |del phi|
Tpvs_sem = Tpvs_sd / np.sqrt(len(Tpvs))
abs_delphis_sem = abs_delphis_sd / np.sqrt(len(abs_delphis))

# For fitting the '|del_phis|' vs 'del T_pvs' relation and finding the slope
slope1, intercept1 = np.polyfit(abs_delphis, Tpvs, 1) #using polyfit for straightline fitting.
Tpvs_fit = ((intercept1) + (slope1 * np.array(abs_delphis))) # drawing the fit line for '|del_phis|' vs 'del T_pvs'
Tpv_diff = Tpvs - Tpvs_fit # Calculate the y-axis residuals for '|del_phis|' vs 'del T_pvs'

# Transmitance of aparture in the absance of a sample (S)
S = (1 - ((slope1/0.406)**(1/0.27)))

# Plotting the graph for '|del_phis|' vs 'del T_pvs'
plt.figure(figsize=(1.618*5, 5))
plt.errorbar(abs_delphis, Tpvs, yerr=abs(Tpv_diff), fmt=".", color='#0C359E', ecolor='green', capsize=5, capthick=0.5, elinewidth=0.5)
plt.title(f'|$\Delta$ $\phi$| vs $\Delta$T$_p$$_v$ for {sample}', fontsize=14)
plt.xlabel('|$\Delta$ $\phi$|')
plt.ylabel('$\Delta$T$_p$$_v$')
plt.plot(abs_delphis, Tpvs_fit, 'r')
plt.text(0.001, 0.95, '$\Delta$T$_p$$_v$ / |$\Delta$ $\phi$| : {:.9f}'.format(slope1), transform=plt.gca().transAxes, fontsize=9, color='#0C359E')
plt.tight_layout()
# plt.savefig(f'{filepath}/{sample} abs_delphi_vs_Tpv.png', dpi=300) # Save the graph
plt.show()


# Power vs Delphi graph

# Fitting the 'pwrs' vs'|del_phis|' relation and finding the slope
slope2, intercept2 = np.polyfit(pwrs, abs_delphis, 1) # using polyfit
delphis_fit = slope2*np.array(pwrs)+intercept2
delphis_diff = abs_delphis - (delphis_fit) # Calculate the y-axis residuals for 'pwrs' vs'|del_phis|'

# Plotting the graph for 'pwrs' vs'|del_phis|'
plt.figure(figsize=(1.618*5, 5))

# color condition for negadive and positive del_phis value
colors = ['red' if d < 0 else '#0C359E' for d in delphis]
for i in range(len(pwrs)):
    if colors[i] == 'red':
        label = 'Negative $\Delta$ $\phi$ values'
    else:
        label = 'Positive $\Delta$ $\phi$ values'
    plt.errorbar(pwrs[i], abs_delphis[i], yerr=abs(delphis_diff[i]), fmt=".", color=colors[i], ecolor='green', capsize=3, capthick=0.5, elinewidth=0.5, label=label)

# Removing duplicate labels in the legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), fontsize=10, loc="upper right")
plt.title(f'Power vs |$\Delta$ $\phi$| for {sample}', fontsize=10)
plt.xlabel('Power in mW')
plt.ylabel('|$\Delta$ $\phi$|')
plt.plot(pwrs, delphis_fit, 'Green', label='fitted line through origin')
plt.text(0.001, 0.95, 'Slope: {:.9f}'.format(slope2), transform=plt.gca().transAxes, fontsize=9, color='#0C359E')
plt.tight_layout()
# plt.savefig(f'{filepath}/{sample} power vs abs_delphi.png', dpi=300) # Save the graph
plt.show()
plt.clf() # Clear data from memory